<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLM_Powered_Text_Adventure_Game_(Python_for_Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import IPython.display as display
from google.colab import userdata
from google.colab import output
import requests
import json
import time # For exponential backoff
import asyncio # Import asyncio
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow asyncio to run inside a running event loop (like Colab's)
nest_asyncio.apply()

# --- Google Colab Specific API Key Setup ---
# This part assumes you are running in Google Colab and have stored your GEMINI API key as a secret.
try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not in Google Colab, or 'userdata' module not found. Please set GOOGLE_API_KEY manually if needed.")
    GOOGLE_API_KEY = None # Fallback for local execution

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20" # Explicitly use the preview model for structured responses

# --- Global Game State ---
game_chat_history = []
current_story_description = "Welcome, adventurer! Click 'Start New Game' to begin your journey."
current_choices = []
game_over_status = False
loading_message = "Generating scenario... Please wait. ✨"
error_message = ""
info_message = ""

# --- LLM API Call Function ---
async def call_llm(prompt_text, max_retries=5, initial_delay=1.0):
    """
    Calls the Gemini LLM API to get simulation parameters.
    Implements exponential backoff for retries.
    """
    global game_chat_history, loading_message, error_message, info_message

    #print("DEBUG: Entering call_llm function.") # DEBUG PRINT
    # Clear previous messages
    error_message = ""
    info_message = ""

    # Check if API key is available
    if not GOOGLE_API_KEY:
        error_message = "Google API Key not available. Cannot call LLM."
        print(f"ERROR: {error_message}") # DEBUG PRINT
        update_game_ui(is_loading=False) # Ensure UI updates with error
        return None

    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{LLM_MODEL_NAME}:generateContent?key={GOOGLE_API_KEY}"
    headers = {"Content-Type": "application/json"}

    # FIX: Add the prompt to history regardless of whether it's an initial prompt or not.
    # The prompt_text is always the user's "turn" in this context.
    game_chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})
    #print(f"DEBUG: Added user prompt to history. Current history length: {len(game_chat_history)}") # DEBUG PRINT

    payload = {
        "contents": game_chat_history,
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": {
                "type": "OBJECT",
                "properties": {
                    "description": {"type": "STRING", "description": "The current scene description"},
                    "choices": {
                        "type": "ARRAY",
                        "items": {"type": "STRING"},
                        "description": "List of possible actions/choices for the player"
                    },
                    "gameOver": {"type": "BOOLEAN", "description": "True if the game has ended"}
                },
                "required": ["description", "choices", "gameOver"]
            }
        }
    }

    retries = 0
    delay = initial_delay

    while retries < max_retries:
        try:
            #print(f"DEBUG: Attempting LLM API call (Retry {retries + 1}/{max_retries})...") # DEBUG PRINT
            # Indicate loading in the UI *before* the actual request
            update_game_ui(is_loading=True)

            response = requests.post(api_url, headers=headers, data=json.dumps(payload))
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            result = response.json()
            #print("DEBUG: LLM API call successful. Parsing response.") # DEBUG PRINT

            if result and result.get("candidates") and result["candidates"][0].get("content") and result["candidates"][0]["content"].get("parts"):
                llm_data_str = result["candidates"][0]["content"].get("parts", [{}])[0].get("text")
                if not llm_data_str:
                    error_message = "LLM response missing expected 'text' part."
                    print(f"ERROR: {error_message}") # DEBUG PRINT
                    return None

                # Ensure it's a string that can be parsed as JSON
                if isinstance(llm_data_str, dict): # Handle cases where it returns dict directly
                    parsed_json = llm_data_str
                    #print("DEBUG: LLM response was already a dict.") # DEBUG PRINT
                else:
                    parsed_json = json.loads(llm_data_str)
                    #print("DEBUG: LLM response successfully parsed from string.") # DEBUG PRINT


                # Add LLM's response to chat history
                game_chat_history.append({"role": "model", "parts": [{"text": llm_data_str}]})
                #print(f"DEBUG: Added LLM response to history. Current history length: {len(game_chat_history)}") # DEBUG PRINT
                return parsed_json
            else:
                error_message = f"LLM response missing expected content structure. Response: {result}"
                print(f"ERROR: {error_message}") # DEBUG PRINT
                if "blocked_reason" in str(result): # Example of checking for content filtering
                     error_message = "LLM response was blocked, potentially due to safety filters."
                     print(f"ERROR: {error_message}") # DEBUG PRINT
                return None

        except requests.exceptions.RequestException as e:
            error_message = f"API request failed (Attempt {retries + 1}/{max_retries}): {e}"
            print(f"ERROR: {error_message}") # DEBUG PRINT
            if response is not None:
                print(f"Response status: {response.status_code}, Body: {response.text}") # DEBUG PRINT
            retries += 1
            if retries < max_retries:
                #print(f"DEBUG: Retrying in {delay} seconds...") # DEBUG PRINT
                await asyncio.sleep(delay)
                delay *= 2 # Exponential backoff
        except json.JSONDecodeError as e:
            error_message = f"Error parsing JSON response from LLM: {e}. Raw response: {response.text if response else 'N/A'}"
            print(f"ERROR: {error_message}") # DEBUG PRINT
            if response and response.text:
                print(f"Raw response snippet: {response.text[:500]}...") # Show snippet # DEBUG PRINT
            return None
        except Exception as e:
            error_message = f"An unexpected error occurred during API call: {e}"
            print(f"ERROR: {error_message}") # DEBUG PRINT
            return None
    error_message = f"Failed to get LLM parameters after {max_retries} retries."
    print(f"ERROR: {error_message}") # DEBUG PRINT
    return None

# --- Game Logic Functions (Python Callbacks) ---

def initial_game_prompt():
    return """You are the game master for a text adventure.
Generate an initial scenario for a space exploration game.
Provide a 'description' (a few sentences, 100-200 words) and a 'choices' array (2-4 distinct options for the player, each a short phrase).
Set 'gameOver' to false.
Example: {"description": "You wake up in a dimly lit spaceship cabin. Alarms blare. The main console shows a critical system failure. What do you do?", "choices": ["Inspect the console", "Head to the escape pod", "Search for crewmates"], "gameOver": false}"""

async def start_new_game():
    global game_chat_history, current_story_description, current_choices, game_over_status, error_message, info_message
    #print("DEBUG: Entered start_new_game function.") # DEBUG PRINT - Simpler print to confirm entry
    game_chat_history = [] # Reset chat history
    game_over_status = False
    error_message = ""
    info_message = ""

    current_story_description = loading_message
    current_choices = []
    update_game_ui(is_loading=True) # Show loading while fetching
    #print("DEBUG: UI updated to loading.") # DEBUG PRINT

    #print("DEBUG: Calling call_llm from start_new_game.") # DEBUG PRINT
    scenario = await call_llm(initial_game_prompt())
    #print(f"DEBUG: Returned from call_llm. Scenario received: {scenario is not None}") # DEBUG PRINT

    if scenario:
        current_story_description = scenario['description']
        current_choices = scenario['choices']
        game_over_status = scenario['gameOver']
        if game_over_status:
            info_message = "Game Over! Click 'Start New Game' to play again."
        print(f"DEBUG: Scenario received. Game Over: {game_over_status}. Choices: {current_choices}") # DEBUG PRINT
    else:
        current_story_description = "Failed to start new game. Please check the Python console for errors."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not retrieve initial scenario."
        print("DEBUG: Failed to get scenario in start_new_game.") # DEBUG PRINT

    update_game_ui(is_loading=False)
    #print("DEBUG: Exiting start_new_game function.") # DEBUG PRINT

async def handle_player_action(player_action):
    global current_story_description, current_choices, game_over_status, error_message, info_message
    print(f"DEBUG: Entering handle_player_action function with action: '{player_action}'") # DEBUG PRINT

    if not player_action.strip():
        error_message = "Please enter your action or click a choice button."
        update_game_ui(is_loading=False) # Ensure loading is off and error shows
        print("DEBUG: Player action was empty.") # DEBUG PRINT
        return

    # Add player's action to history for display
    current_story_description += f'<p class="text-blue-300 italic mt-2">You chose: "{player_action}"</p>'
    update_game_ui(is_loading=True) # Show loading while fetching
    print("DEBUG: UI updated to loading for action.") # DEBUG PRINT


    game_prompt = f"""The player chose: "{player_action}".
Given the previous story: "{game_chat_history[-2]['parts'][0]['text'] if len(game_chat_history) >= 2 else ''}", continue the adventure.
Provide a 'description' (a few sentences, 100-200 words) that advances the plot based on the player's choice.
Provide a 'choices' array (2-4 distinct options for the player, each a short phrase).
Set 'gameOver' to true if the story reaches a clear end (success or failure), otherwise false.
If the game ends, the description should clearly state the outcome (e.g., "You successfully repaired the ship! Game Over.")."
"""
    print(f"DEBUG: Calling LLM with game prompt for action: '{player_action}'") # DEBUG PRINT
    scenario = await call_llm(game_prompt)
    print(f"DEBUG: Returned from call_llm for action. Scenario received: {scenario is not None}") # DEBUG PRINT

    if scenario:
        current_story_description = scenario['description']
        current_choices = scenario['choices']
        game_over_status = scenario['gameOver']
        if game_over_status:
            info_message = "Game Over! Click 'Start New Game' to play again."
        print(f"DEBUG: Scenario received for action. Game Over: {game_over_status}. Choices: {current_choices}") # DEBUG PRINT
    else:
        current_story_description = "An error occurred while continuing the story. Please try again or start a new game."
        current_choices = []
        game_over_status = True # End game on error
        error_message = "Error: Could not advance scenario."
        print("DEBUG: Failed to advance scenario in handle_player_action.") # DEBUG PRINT


    update_game_ui(is_loading=False)
    #print("DEBUG: Exiting handle_player_action function.") # DEBUG PRINT


# --- UI Generation Function ---
def update_game_ui(is_loading=False):
    global current_story_description, current_choices, game_over_status, loading_message, error_message, info_message

    choices_html = ""
    if not game_over_status and not is_loading:
        for i, choice in enumerate(current_choices):
            # Pre-process the choice to escape single quotes for JavaScript
            # A single quote ' needs to become \' in JavaScript.
            escaped_choice_for_js = choice.replace("'", "\\'")

            choices_html += f"""
            <button onclick="handleChoiceJS('{escaped_choice_for_js}')"
                    class="bg-slate-600 hover:bg-slate-700 text-white py-2 px-4 rounded-lg transition duration-200 focus:outline-none focus:ring-2 focus:ring-slate-500">
                {i + 1}. {choice}
            </button>
            """

    message_area_class = "message-box hidden"
    message_content = ""
    if error_message:
        message_area_class = "message-box bg-red-700"
        message_content = error_message
    elif info_message:
        message_area_class = "message-box bg-blue-700"
        message_content = info_message

    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>LLM Text Adventure</title>
        <script src="https://cdn.tailwindcss.com"></script>
        <style>
            body {{
                font-family: 'Inter', sans-serif;
                background-color: #0f172a; /* Dark blue-gray */
                color: #e2e8f0; /* Light gray text */
            }}
            .container {{
                max-width: 800px;
                margin: 2rem auto;
                padding: 1.5rem;
                background-color: #1e293b; /* Slightly lighter blue-gray */
                border-radius: 0.75rem; /* Rounded corners */
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }}
            .message-box {{
                background-color: #334155; /* Even lighter for messages */
                padding: 1rem;
                border-radius: 0.5rem;
                margin-top: 1rem;
                color: #cbd5e1;
            }}
            @media (max-width: 640px) {{
                .container {{
                    margin: 1rem;
                    padding: 1rem;
                }}
            }}
        </style>
    </head>
    <body class="flex items-center justify-center min-h-screen">
        <div class="container space-y-4">
            <h1 class="text-3xl font-bold text-center text-white mb-6">🌌 LLM Space Adventure 🚀</h1>

            <div id="loading" class="text-center text-yellow-400 font-semibold {'hidden' if not is_loading else ''}">
                <p>{loading_message}</p>
            </div>

            <div id="story-display" class="bg-slate-700 p-6 rounded-lg text-lg leading-relaxed border border-slate-600">
                {current_story_description}
            </div>

            <div id="choices-display" class="flex flex-wrap gap-3 mt-4">
                {choices_html if not is_loading else ''}
            </div>

            <div class="flex flex-col sm:flex-row gap-3 mt-4">
                <input type="text" id="user-input" placeholder="Type your action or choice number..."
                       class="flex-grow p-3 rounded-lg bg-slate-800 text-white border border-slate-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                       {'disabled' if game_over_status or is_loading else ''}>
                <button id="submit-button"
                        class="bg-blue-600 hover:bg-blue-700 text-white font-semibold py-3 px-6 rounded-lg shadow-md transition duration-300 ease-in-out transform hover:scale-105 focus:outline-none focus:ring-2 focus:ring-blue-500"
                        onclick="submitActionFromInput()"
                        {'disabled' if game_over_status or is_loading else ''}>
                    Submit Action
                </button>
                <button id="start-new-game-button"
                        class="bg-green-600 hover:bg-green-700 text-white font-semibold py-3 px-6 rounded-lg shadow-md transition duration-300 ease-in-out transform hover:scale-105 focus:outline-none focus:ring-2 focus:ring-green-500"
                        onclick="startNewGameJS()"
                        {'disabled' if is_loading else ''}>
                    Start New Game
                </button>
            </div>

            <div id="message-area" class="{message_area_class}">
                {message_content}
            </div>
        </div>

        <script>
            // JavaScript functions to call Python callbacks
            async function startNewGameJS() {{
                console.log("DEBUG JS: startNewGameJS called"); // DEBUG PRINT
                await google.colab.kernel.invokeFunction('start_new_game', [], {{}});
                console.log("DEBUG JS: startNewGameJS invokeFunction returned"); // DEBUG PRINT
            }}

            async function handleChoiceJS(choice) {{
                console.log("DEBUG JS: handleChoiceJS called with choice:", choice); // DEBUG PRINT
                await google.colab.kernel.invokeFunction('handle_player_action', [choice], {{}});
                console.log("DEBUG JS: handleChoiceJS invokeFunction returned"); // DEBUG PRINT
            }}

            async function submitActionFromInput() {{
                const userInput = document.getElementById('user-input');
                const action = userInput.value;
                if (action) {{
                    console.log("DEBUG JS: submitActionFromInput called with action:", action); // DEBUG PRINT
                    await google.colab.kernel.invokeFunction('handle_player_action', [action], {{}});
                     console.log("DEBUG JS: submitActionFromInput invokeFunction returned"); // DEBUG PRINT
                }} else {{
                    console.log("DEBUG JS: submitActionFromInput called with empty action."); // DEBUG PRINT
                }}
                userInput.value = ''; // Clear input after submission
            }}

            // Event listener for Enter key on input field
            document.getElementById('user-input').addEventListener('keypress', function(e) {{
                if (e.key === 'Enter') {{
                    submitActionFromInput();
                }}
            }});
             console.log("DEBUG JS: Script loaded."); // DEBUG PRINT
        </script>
    </body>
    </html>
    """
    # Clear the cell and display new HTML content
    display.clear_output(wait=True)
    display.display(display.HTML(html_content))

# --- Main Execution ---

# Define synchronous wrappers to call the async functions
def sync_start_new_game():
    asyncio.run(start_new_game())

def sync_handle_player_action(player_action):
    asyncio.run(handle_player_action(player_action))

# Register Python functions as callbacks callable from JavaScript
# These must be called after the functions are defined.
output.register_callback('start_new_game', sync_start_new_game)
output.register_callback('handle_player_action', sync_handle_player_action)

print("DEBUG: Callbacks registered.") # DEBUG PRINT

# Initial display of the game UI
update_game_ui()
#print("DEBUG: Initial UI displayed.") # DEBUG PRINT